# Data analysis

In [1]:
import pandas as pd
import numpy as np

In [2]:
import matplotlib.pyplot as plt
%matplotlib notebook

In [3]:
csvfile = "meetings_complete_clean.csv"

In [4]:
df = pd.read_csv(csvfile)

In [5]:
df.head()

,Name,Day,Time,Video,Desc,Categories,Email,Phone,Time_dt,Time_del,...,Label_Game,Label_Blind / Visually Impaired,Label_Deaf / Hard of Hearing,Label_Email,Label_Professionals,Label_Meditation,Label_Seniors,Label_Newcomer,Label_24/7,Zoom
0,1 Hole in the Sky AA Meetings: AA Secular Dail...,Tuesday,5:00 pm,https://zoom.us/j/414948190,Tuesday: AA Secular Daily Reflections 3:00pm P...,"Audio,Open,Video",NaN,NaN,17:00:00,0 days 17:00:00.000000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
1,AA Friends,Tuesday,5:00 pm,https://aafriends.webs.com/,AA Friends currently holds three online meetin...,"Forum,Open",aafriends@usa.com,NaN,17:00:00,0 days 17:00:00.000000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
2,AA New Beginnings,Tuesday,5:00 pm,https://us02web.zoom.us/j/5662326551,"A daily meeting that focuses on experience, st...","Audio,Big Book,Discussion,Open,Video",AANewBeginnings230@gmail.com,NaN,17:00:00,0 days 17:00:00.000000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
3,Agnes Water,Tuesday,5:00 pm,https://us02web.zoom.us/j/8983968244,PW billw,"Big Book,Discussion,Speaker,Video",agneswaterxa@gmail.com,NaN,17:00:00,0 days 17:00:00.000000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
4,Beginners AA meeting,Tuesday,5:00 pm,https://us02web.zoom.us/j/88198562544?pwd=wfzm...,"Zoom Passcode, if any: serenity","Big Book,Discussion,Open,Speaker,Video",liaazoom2020@gmail.com,NaN,17:00:00,0 days 17:00:00.000000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True


In [6]:
df.shape

(4155, 37)

In [7]:
df.columns

Index(['Name', 'Day', 'Time', 'Video', 'Desc', 'Categories', 'Email', 'Phone',
       'Time_dt', 'Time_del', 'Label_Audio', 'Label_Open', 'Label_Video',
       'Label_Forum', 'Label_Big Book', 'Label_Discussion', 'Label_Speaker',
       'Label_Closed', 'Label_Telephone', 'Label_Women',
       'Label_Loners / Isolationists', 'Label_Chat', 'Label_LGBTQIAA+',
       'Label_Young People', 'Label_Men', 'Label_Secular',
       'Label_Steps / Traditions', 'Label_Game',
       'Label_Blind / Visually Impaired', 'Label_Deaf / Hard of Hearing',
       'Label_Email', 'Label_Professionals', 'Label_Meditation',
       'Label_Seniors', 'Label_Newcomer', 'Label_24/7', 'Zoom'],
      dtype='object')